In [1]:
#!/user/bin/env python
import sys
import string
import csv
import math
import re
import xlwt
import datetime


In [2]:
file = "sparkFiles/spark-events/app-20190904192037-0001"
def getStages(filename):
    stage_count = 0
    with open(filename) as openLog:
        for line in openLog:
            if '"Event":"SparkListenerStageCompleted"' in line:
                if '"Stage ID":' in line:
                    #print(line)
                    stage_count = stage_count+1
    
    #print(stage_count)

    return stage_count

getStages(file)

38

In [3]:
def getTotalTasks(filename, stage):
    total = 0
    stage_check = '"Stage ID":'+str(stage)
    with open(filename) as openLog:
        for line in openLog:
                if '"Event":"SparkListenerStageSubmitted"' and stage_check in line:
                    if '"Number of Tasks":' in line:
                        partB = line.split('"Number of Tasks":')[1]
                        partB = partB.split(",")
                        tasks = partB[0] 
                        total = int(tasks)

        return total
print(getTotalTasks(file,0))

1


In [4]:

def getMeanTaskTime(filename, stage):
        
        ttet = 0.0
        mtet = 0.0
        stage_check = '"Stage ID":'+str(stage)
        count = getTotalTasks(file, stage)
        with open(filename) as openLog:
                for line in openLog:
                        if '"Event":"SparkListenerStageCompleted"' in line:
                                if stage_check in line:
                                        #Executor Run Time
                                        partA = line.split('executorRunTime')[1]
                                        partA = partA.split('"Value":')
                                        tet = int(partA[1].split(',')[0])
                                        ttet = ttet + int(tet)
        mtet = math.ceil(ttet/count)
        mtet = mtet/1000
        mtet = mtet/60
        
        return mtet

print(getMeanTaskTime(file, 0))

0.009733333333333333


In [5]:
def getMeanTaskDeserializationTime(filename,stage):
    ttdt = 0.0
    mtdt = 0.0
    stage_check = '"Stage ID":'+str(stage)
    count = getTotalTasks(file, stage)
    with open(filename) as openLog:
        for line in openLog:
            if '"Event":"SparkListenerTaskEnd"' in line:
                if stage_check in line:
                    #Executor Deserialize CPU Time
#                     partA = line.split('executorDeserializeCpuTime')[1]
#                     partA = partA.split('"Value":')
#                     tdt = int(partA[1].split(',')[0])
#                     converted_tdt = float(tdt)/1000000000
                    #Executor Deserialize Time
                    partB = line.split('"Executor Deserialize Time":')[1]
                    partB = partB.split(',')
                    tdt = int(partB[0])
                    converted_tdt = float(tdt)/1000
                    ttdt = ttdt + converted_tdt


        mtdt = ttdt/count
        mtdt = mtdt

        
    #print(ttdt)
    return mtdt


print(getMeanTaskDeserializationTime(file,34))

0.0067649999999999984


In [6]:
# file = "./sparkFiles/spark-events/app-20190225134146-0001"
# file = "./sparkLogsData/app-20190130101035-0000"
def getMeanShuffleReadTime(filename, stage):
        reference_time = 1548868837392
        tsrt = 0.0
        msrt = 0.0
        stage_check = '"Stage ID":'+str(stage)
        count = 0
        with open(filename) as openLog:
                for line in openLog:
                        if '"Event":"SparkListenerTaskEnd"' in line:
                                if stage_check in line:
                                        partA, partB = line.split('"Fetch Wait Time":')
                                        partB = partB.split(',')
                                        srt = int(partB[0])
                                        tsrt = tsrt + int(srt)
                                        count = count + 1
        msrt = math.ceil(tsrt/count)
        msrt = msrt/1000
        msrt = msrt/60
        
        return msrt

print(getMeanShuffleReadTime(file, 25))

0.0


In [7]:
file = "sparkFiles/spark-events/app-20200416121042-0000"
def getMeanGCTime(filename, stage):
        reference_time = 1548868837392
        tgct = 0.0
        mgct = 0.0
        stage_check = '"Stage ID":'+str(stage)
        task_success_check = '{"Reason":"Success"}'
        count = 0
        with open(filename) as openLog:
                for line in openLog:
                        if '"Event":"SparkListenerTaskEnd"' in line:
                                if stage_check in line:
                                    if task_success_check in line:
                                        partA, partB = line.split('"JVM GC Time":')
                                        partB = partB.split(',')
                                        gct = int(partB[0])
                                        tgct = tgct + int(gct)
                                        count = count + 1
        #print(tgct)
        mgct = math.ceil(tgct/count)
        mgct = mgct/1000
        mgct = mgct/60
        return mgct

print(getMeanGCTime(file,25))


0.005966666666666666


In [8]:
def getMeanSchedulerDelay(filename, stage):
        msd = 0
        schedulerDelay = 0
        count = 0
        stage_check = '"Stage ID":'+str(stage)
        task_success_check = '{"Reason":"Success"}'
        with open(filename) as openLog:
                for line in openLog:
                        if '"Event":"SparkListenerTaskEnd"':
                                if stage_check in line:
                                    if task_success_check in line:
                                        start_time = line.split('"Launch Time":')[1]
                                        start_time = start_time.split(',')[0]
                                        finish_time = line.split('"Finish Time":')[1]
                                        finish_time = finish_time.split(',')[0]
                                        
                                        TimestampUtc = "\/Date(" + start_time + ")\/"
                                        TimestampUtc = re.split('\(|\)', TimestampUtc)[1][:10]

    
                                        date_s = datetime.datetime.fromtimestamp(int(TimestampUtc))
                                        #print (date_s)
    
                                        TimestampUtc = "\/Date(" + finish_time + ")\/"
                                        TimestampUtc = re.split('\(|\)', TimestampUtc)[1][:10]

                                        date_f = datetime.datetime.fromtimestamp(int(TimestampUtc))
                                        #print (date_f)

                                        duration = date_f - date_s
                                        duration = str(duration)
    
                                        duration = duration.split(':')
    
                                        task_duration_in_secs = (60*60*int(duration[0])) + (60*int(duration[1])) + int(duration[2])

                                        executorRunTime = line.split('"Executor Run Time":')
                                        executorRunTime = executorRunTime[1].split(',')[0]
                                        
                                        executorDeserializeTime = line.split('"Executor Deserialize Time":')
                                        executorDeserializeTime = int(executorDeserializeTime[1].split(',')[0])
                                        
                                        resultSerializationTime = line.split('"Result Serialization Time":')
                                        resultSerializationTime = int(resultSerializationTime[1].split(',')[0])
                                        
                                        gettingResultTime = line.split('"Getting Result Time":')
                                        gettingResultTime = int(gettingResultTime[1].split(',')[0])
                                        
                                        executorOverhead = float(executorDeserializeTime)/1000 + float(resultSerializationTime)/1000

                                        delay = task_duration_in_secs - float(executorRunTime)/1000 - executorOverhead \
                                                - float(gettingResultTime)/1000000000

                                        if delay <= 0:
                                            scehdulerDelay = schedulerDelay + 0
                                        else:
                                            schedulerDelay = schedulerDelay + delay
                                            
                                        count = count+1
                      
        msd = schedulerDelay/count
        msd = msd/60
        return msd
    
print(getMeanSchedulerDelay(file,25))

0.002965277777777779


In [10]:
def getMeanShuffleWriteTime(filename, stage):
        tswt = 0.0
        mswt = 0.0
        stage_check = '"Stage ID":'+str(stage)
        count = 0
        task_success_check = '{"Reason":"Success"}'
        with open(filename) as openLog:
                for line in openLog:
                        if '"Event":"SparkListenerTaskEnd"' in line:
                                if stage_check in line:
                                    if task_success_check in line:
                                        partA, partB = line.split('"Shuffle Write Time":')
                                        partB = partB.split(',')
                                        swt = int(partB[0])

                                        tswt = tswt + int(swt)
                                        count = count + 1
                                        
        print(tswt)
        mswt = math.ceil(tswt/count)
        mswt = mswt/1000000000
        mswt = mswt/60
        return mswt

print(getMeanShuffleWriteTime(file, 0))


0.0
0.0


In [11]:

def getMedianSchedulerDelay(filename, stage):
        msd = 0.0
        schedulerDelay = []
        count = 0
        stage_check = '"Stage ID":'+str(stage)
        task_success_check = '{"Reason":"Success"}'
        with open(filename) as openLog:
                for line in openLog:
                        if '"Event":"SparkListenerTaskEnd"':
                                if stage_check in line:
                                    if task_success_check in line:
                                        
                                        start_time = line.split('"Launch Time":')[1]
                                        start_time = start_time.split(',')[0]
                                        finish_time = line.split('"Finish Time":')[1]
                                        finish_time = finish_time.split(',')[0]
                                        
                                        TimestampUtc = "\/Date(" + start_time + ")\/"
                                        TimestampUtc = re.split('\(|\)', TimestampUtc)[1][:10]

    
                                        date_s = datetime.datetime.fromtimestamp(int(TimestampUtc))
                                        #print (date_s)
    
                                        TimestampUtc = "\/Date(" + finish_time + ")\/"
                                        TimestampUtc = re.split('\(|\)', TimestampUtc)[1][:10]

                                        date_f = datetime.datetime.fromtimestamp(int(TimestampUtc))
                                        #print (date_f)

                                        duration = date_f - date_s
                                        duration = str(duration)
    
                                        duration = duration.split(':')
    
                                        task_duration_in_secs = (60*60*int(duration[0])) + (60*int(duration[1])) + int(duration[2])
                                        
                                        executorRunTime = line.split('"Executor Run Time":')
                                        executorRunTime = executorRunTime[1].split(',')[0]
                                        
                                        executorDeserializeTime = line.split('"Executor Deserialize Time":')
                                        executorDeserializeTime = executorDeserializeTime[1].split(',')[0]
                                        
                                        resultSerializationTime = line.split('"Result Serialization Time":')
                                        resultSerializationTime = resultSerializationTime[1].split(',')[0]
                                        
                                        gettingResultTime = line.split('"Getting Result Time":')
                                        gettingResultTime = gettingResultTime[1].split(',')[0]
                                        
                                        executorOverhead = float(executorDeserializeTime)/1000 + float(resultSerializationTime)/1000
                                        
                                        delay = task_duration_in_secs - float(executorRunTime)/1000 - executorOverhead \
                                                - float(gettingResultTime)/1000000000
                                        
                                        schedulerDelay.append(delay)
                                        count = count + 1
        schedulerDelay.sort()     
        if count == 1:
            return schedulerDelay[0]/60
                
        if count%2 != 0:
            index  = (count + 1)/2
            index = int(index)
            msd = schedulerDelay[index-1]
            if msd <= 0:
                return 0
            else:
                return msd/60
        else:
            index = count/2
            index = int(index)
            msd = (schedulerDelay[index-1] + schedulerDelay[index])/2
            if msd <= 0:
                return 0
            else:
                return msd/60
                 
print(getMedianSchedulerDelay(file,0))                

0.007249999999999999


In [12]:

def getMedianTaskTime(filename, stage):
        reference_time = 1548868837392
        ttet = []
        mtet = 0.0
        stage_check = '"Stage ID":'+str(stage)
        count = 0
        task_success_check = '{"Reason":"Success"}'
        with open(filename) as openLog:
                for line in openLog:
                        if '"Event":"SparkListenerTaskEnd"' in line:
                                if stage_check in line:
                                    if task_success_check in line:
                                        #Executor Run Time
                                        partA, partB = line.split('"Executor Run Time":')
                                        partB = partB.split(',')
                                        tet = float(partB[0])
                                        tet = tet/1000
                                        #Executor CPU Time
#                                         partA, partB = line.split('"Executor CPU Time":')
#                                         partB = partB.split(',')
#                                         tet = tet + int(partB[0])

                                        ttet.append(tet)
                                        count = count + 1                
        ttet.sort()
        if count == 1:
                return ttet[0]/60
        
        if count%2 != 0:
                index  = (count + 1)/2
                index = int(index)
                median = ttet[index-1]/60
                return median
        else:
                index = count/2
                index = int(index)
                median = ((ttet[index-1] + ttet[index])/2)/60
                return median

print(getMedianTaskTime(file, 27))



0.0002666666666666667


In [10]:

def getMedianTaskDeserializationTime(filename, stage):
        ttdt = []
        mtdt = 0.0
        stage_check = '"Stage ID":'+str(stage)
        count = 0
        task_success_check = '{"Reason":"Success"}'
        with open(filename) as openLog:
                for line in openLog:
                        if '"Event":"SparkListenerTaskEnd"' in line:
                                if stage_check in line:
                                    if task_success_check in line:
                                        #Executor Desrialize Time
                                        partA, partB = line.split('"Executor Deserialize Time":')
                                        partB = partB.split(',')
                                        tdt = int(partB[0])
                                        converted_tdt = float(tdt)/1000
                                        #Executor Deserialize CPU Time
#                                         partA, partB = line.split('"Executor Deserialize CPU Time":')
#                                         partB = partB.split(',')
#                                         tdt = int(partB[0])
#                                         converted_tdt = converted_tdt + float(tdt)/1000000000

                                        ttdt.append(converted_tdt)
                                        count = count + 1

        ttdt.sort()
        if count == 1:
                return ttdt[0]/60
        if count%2 != 0:
                index  = (count + 1)/2
                index = int(index)
                median = ttdt[index-1]
                return median/60
        else:
                index = count/2
                index = int(index)
                median = (ttdt[index-1] + ttdt[index])/2
                return median/60

print(getMedianTaskDeserializationTime(file, 32))

5e-05


In [13]:
def getMedianShuffleReadTime(filename, stage):
        tsrt = []
        stage_check = '"Stage ID":'+str(stage)
        count = 0
        task_success_check = '{"Reason":"Success"}'
        with open(filename) as openLog:
                for line in openLog:
                        if '"Event":"SparkListenerTaskEnd"' in line:
                                if stage_check in line:
                                    if task_success_check in line:
                                        partA, partB = line.split('"Fetch Wait Time":')
                                        partB = partB.split(',')
                                        srt = int(partB[0])
                                        srt = srt/1000
                                        tsrt.append(srt)
                                        count = count + 1
        tsrt.sort()
        if count == 1:
                return tsrt[0]/60 

        if count%2 != 0:
                index  = (count + 1)/2
                index = int(index)
                median = tsrt[index-1]
                return median/60
        else:
                index = count/2
                index = int(index)
                median = (tsrt[index-1] + tsrt[index])/2
                return median/60

print(getMedianShuffleReadTime(file, 0))


0.0


In [15]:

def getMedianShuffleWriteTime(filename,stage):
        tswt = []
        mswt = 0.0
        stage_check = '"Stage ID":'+str(stage)
        count = 0
        task_success_check = '{"Reason":"Success"}'
        with open(filename) as openLog:
                for line in openLog:
                        if '"Event":"SparkListenerTaskEnd"' in line:
                                if stage_check in line:
                                    if task_success_check in line:
                                        partA, partB = line.split('"Shuffle Write Time":')
                                        partB = partB.split(',')
                                        swt = int(partB[0])
                                        swt = swt/1000000000
                                        tswt.append(swt)
                                        count = count + 1
                                        
        if count == 1:
            return tswt[0]/60 
        if count%2 != 0:
            index  = (count + 1)/2
            index = int(index)
            median = tswt[index-1]
            return median/60
        else:
            index = count/2
            index = int(index)
            median = (tswt[index-1] + tswt[index])/2
            return median/60

print(getMedianShuffleWriteTime(file,32))

0.002402414041666667


In [48]:
def getMedianGCTime(filename, stage):
        reference_time = 1548868837392
        tgct = []
        stage_check = '"Stage ID":'+str(stage)
        count = 0
        task_success_check = '{"Reason":"Success"}'
        with open(filename) as openLog:
                for line in openLog:
                        if '"Event":"SparkListenerTaskEnd"' in line:
                                if stage_check in line:
                                    if task_success_check in line:
                                        partA, partB = line.split('"JVM GC Time":')
                                        partB = partB.split(',')
                                        gct = int(partB[0])
                                        gct = float(gct)/1000
                                        tgct.append(gct)
                                        count = count + 1
        tgct.sort()
        if count == 1:
                return tgct[0]/60 
        elif count%2 != 0:
                index  = (count + 1)/2
                index = int(index)
                median = tgct[index-1]
                return median/60
        else:
                index = count/2
                index = int(index)
                median = (tgct[index-1] + tgct[index])/2
                return median/60

print(getMedianGCTime(file, 27))


0.0


In [19]:
def getPercentFailedTasks(filename, stage):
        failed_tasks = 0
        total_tasks = getTotalTasks(filename, stage)
        stage_check = '"Stage ID":'+str(stage)
        task_success_check = '{"Reason":"Success"}'
        with open(filename) as openLog:
                for line in openLog:
                        if '"Event":"SparkListenerTaskEnd"' in line:
                                if stage_check in line:
                                    if task_success_check not in line:
                                        failed_tasks = failed_tasks + 1
                                        
        
        percent_failed_tasks = failed_tasks/total_tasks
        return percent_failed_tasks

getPercentFailedTasks(file,1)

0.0

In [17]:
def getTaskCompletionState(filename, stage):
    reference_time = 1548868837392
    total_tasks = getTotalTasks(filename, stage)
    tasks_completed = 0
    stage_check = '"Stage ID":'+str(stage)
    task_success_check = '{"Reason":"Success"}'
    with open(filename) as openLog:
        for line in openLog:
            if stage_check in line:
                if '"Event":"SparkListenerTaskEnd"' and task_success_check in line:
                    tasks_completed = tasks_completed + 1
    if tasks_completed == total_tasks:
        return 1
    else:
        return 0
    
print (getTaskCompletionState("./sparkFiles/spark-events/app-20190221155750-0001", 0))
# app-20190225134146-0001

1


In [22]:
file = "./sparkLogsData/app-20190130101035-0000"
def getStageDuration(filename, stage):
    stage_duration = 0
    stage_check = '"Stage ID":' + str(stage) + ','
    start_time = ''
    finish_time = ''
    with open(filename) as openLog:
            for line in openLog:
                if '"Event":"SparkListenerStageCompleted"' in line:
                    if stage_check in line:
                        start_time = line.split('"Submission Time":')[1]
                        start_time = start_time.split(',')[0]
                        finish_time = line.split('"Completion Time":')[1]
                        finish_time = finish_time.split(',')[0]
                        break
    #print (start_time)
    TimestampUtc = "\/Date(" + start_time + ")\/"
    TimestampUtc = re.split('\(|\)', TimestampUtc)[1][:10]

    
    date_s = datetime.datetime.fromtimestamp(int(TimestampUtc))
    #print (date_s)
    
    TimestampUtc = "\/Date(" + finish_time + ")\/"
    TimestampUtc = re.split('\(|\)', TimestampUtc)[1][:10]

    date_f = datetime.datetime.fromtimestamp(int(TimestampUtc))
    #print (date_f)

    duration = date_f - date_s
    duration = str(duration)
    
    duration = duration.split(':')
    
    duration_in_secs = (60*60*int(duration[0])) + (60*int(duration[1])) + int(duration[2])
    
    duration_in_mins = duration_in_secs/60
    
    return duration_in_mins

print(getStageDuration(file, 0))

9.983333333333333


In [20]:
#calculate all mean and median delays and task times

#all times are in minutes

filename = "./sparkLogsData/app-20190130101035-0000"

stages = getStages(filename)
meanTaskTimePerStage = []
medianTaskTimePerStage = []

meanSchedulerDelayPerStage = []
meanTaskDeserializationTimePerStage = []
meanShuffleReadTimePerStage = []
meanShuffleWriteTimePerStage = []
meanGCTimePerStage = []

medianSchedulerDelayPerStage = []
medianTaskDeserializationTimePerStage = []
medianShuffleReadTimePerStage = []
medianShuffleWriteTimePerStage = []
medianGCTimePerStage = []

stageDurations = []

for stage in range(stages):
    stageTime = getStageDuration(filename, stage)
    stageDurations.append(stageTime)
    
    meanTaskTime = getMeanTaskTime(filename,stage)
    meanTaskDeserializationTime =  getMeanTaskDeserializationTime(filename, stage)
    meanShuffleReadTime = getMeanShuffleReadTime(filename, stage) 
    meanShuffleWriteTime = getMeanShuffleWriteTime(filename, stage)
    meanGCTime = getMeanGCTime(filename, stage)
    meanSchedulerDelay = getMeanSchedulerDelay(filename, stage)
    
    medianTaskTime =  getMedianTaskTime(filename, stage)
    medianTaskDeserializationTime = getMedianTaskDeserializationTime(filename, stage)
    medianShuffleReadTime = getMedianShuffleReadTime(filename, stage)
    medianShuffleWriteTime = getMedianShuffleWriteTime(filename, stage)
    medianGCTime = getMedianGCTime(filename, stage)
    medianSchedulerDelay = getMedianSchedulerDelay(filename, stage)
    
    meanTaskTimePerStage.append(meanTaskTime)
    medianTaskTimePerStage.append(medianTaskTime)
    
    meanSchedulerDelayPerStage.append(meanSchedulerDelay)
    meanTaskDeserializationTimePerStage.append(meanTaskDeserializationTime)
    meanShuffleReadTimePerStage.append(meanShuffleReadTime)
    meanShuffleWriteTimePerStage.append(meanShuffleWriteTime)
    meanGCTimePerStage.append(meanGCTime)

    medianSchedulerDelayPerStage.append(medianSchedulerDelay)
    medianTaskDeserializationTimePerStage.append(medianTaskDeserializationTime)
    medianShuffleReadTimePerStage.append(medianShuffleReadTime)
    medianShuffleWriteTimePerStage.append(medianShuffleWriteTime)
    medianGCTimePerStage.append(medianGCTime)

print("All Times are in Minutes\n")
print("Overall Stage Times")
print(stageDurations)

print("\nMean Task Times Per Stage")
print(meanTaskTimePerStage)
print("Median Task Times Per Stage")
print(medianTaskTimePerStage)

print("\nMean Scheduler Delay Per Stage")
print(meanSchedulerDelayPerStage)
print("Median Scheduler Delay Per Stage")
print(medianSchedulerDelayPerStage)

print("\nMean Task Deserialization Dealy Per Stage")
print(meanTaskDeserializationTimePerStage)
print("Median Task Deserailization Delay Per Stage")
print(medianTaskDeserializationTimePerStage)

print("\nMean Shuffle Read Time Per Stage")
print(meanShuffleReadTimePerStage)
print("Median Shuffle Read Time Per Stage")
print(medianShuffleReadTimePerStage)


print("\nMean Shuffle Write Time Per Stage")
print(meanShuffleWriteTimePerStage)
print("Median Shuffle Write Time Per Stage")
print(medianShuffleWriteTimePerStage)


print("\nMean GC Time Per Stage")
print(meanGCTimePerStage)
print("Median GC Time Per Stage")
print(medianGCTimePerStage)


All Times are in Minutes

Overall Stage Times
[9.983333333333333, 0.016666666666666666]

Mean Task Times Per Stage
[5.213066666666666, 0.006633333333333334]
Median Task Times Per Stage
[5.804150000000001, 0.0067]

Mean Scheduler Delay Per Stage
[0.0, 0.0]
Median Scheduler Delay Per Stage
[0, 0]

Mean Task Deserialization Dealy Per Stage
[0.08004740576666668, 0.0038125453166666665]
Median Task Deserailization Delay Per Stage
[0.014256727583333333, 0.0006037435833333334]

Mean Shuffle Read Time Per Stage
[0.0, 1.6666666666666667e-05]
Median Shuffle Read Time Per Stage
[0.0, 0.0]

Mean Shuffle Write Time Per Stage
[0.00039824688333333334, 0.0]
Median Shuffle Write Time Per Stage
[0.0004622724416666667, 0.0]

Mean GC Time Per Stage
[1.6203, 0.0]
Median GC Time Per Stage
[1.968625, 0.0]
